In [ ]:
import pandas as pd
import openpyxl
from pandas import ExcelWriter

In [ ]:
%store -r tarjetas
%store -r bancos

In [ ]:
tarjeta_nativa = tarjetas.loc [tarjetas['Cod Marca Producto']== 'Mastercard'] #Filtro por la Tarjeta Mastercard
tarjeta_nativa = tarjeta_nativa.drop(tarjeta_nativa.index[(tarjeta_nativa["Cod Comercio"] == 4547438)],axis=0) # Sacar ese Cod de Comercio
tarjeta_nativa = tarjeta_nativa.loc[:, (tarjeta_nativa!= 0).any(axis=0)]

In [ ]:
tarjeta_nativa["PERCEPCION IVA R.G. 2408  1,50/3,00 %"] = tarjeta_mastercard[
                  ['PERCEPCION IVA R.G. 2408   1,50 %', 'PERCEPCION IVA R.G. 2408   3,00 %']].sum(axis=1, min_count=1)
tarjeta_nativa['VENTAS'] = tarjeta_nativa['VENTAS C/DESCUENTO CONTADO'] + tarjeta_nativa['VENTAS C/DTO CUOTAS FINANC. OTORG.']
tarjeta_nativa['RETENCION ING.BRUTOS SIRTAC'] = (tarjeta_nativa['RETENCION ING.BRUTOS SIRTAC']
                                                  + tarjeta_nativa['AJUSTE SIRTAC OCT. 1RA QUIN'])                  
tarjeta_nativa["TOTAL DEDUCCIONES"] = tarjeta_nativa[["TOTAL DEDUCCIONES", "TOTAL PAGOS DE COMERCIOS",'CARGOS DE ENTIDAD']
                                                             ].sum(axis=1, min_count=1)
tarjeta_nativa['IVA 21%'] = tarjeta_nativa[['IVA CRED.FISC.COMERCIO S/ARANC 21,00%','IVA PROMO CUOTAS AHORA 12/18 - 21,00%'
                              , 'IVA RI SIST CUOTAS','IVA RI CRED.FISC.COMERCIO S/DTO F.OTORG']].sum(axis=1, min_count=1)
tarjeta_nativa['ARANCEL'] = tarjeta_nativa['ARANCEL'] + tarjeta_nativa['CARGO  SISTEMA CUOTAS MENS']
tarjeta_nativa['DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18'] = (tarjeta_nativa['DESCUENTO FINANC.OTORG. CUOTAS']
                                                                              + tarjeta_nativa['PROMO CUOTAS AHORA 12/18'])
tarjeta_nativa['Iva 10,5%'] = (tarjeta_nativa['IVA PROMO CUOTAS AHORA 12/18 - 10,50%'] 
                               + tarjeta_nativa['IVA CRED.FISC.COM.L.25063 S/DTO F.OTOR 10,50'])
tarjeta_nativa = tarjeta_nativa.reindex(columns=['Fecha de Pago', 'Cod Marca Producto', 'Cod Comercio',
       'Nro Liquidacion','VENTAS', 'ARANCEL', 'IVA 21%', 'DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18',
       'Iva 10,5%', 'RETENCION IVA','PER B.A.I.BR.DN.01/04', 'RETENCION IMP.GANANCIAS','RETENCION ING.BRUTOS',
       'RETENCION ING.BRUTOS SIRTAC','PERCEPCION IVA R.G. 2408   3,00 %','REEMBOLSOS DE ENTIDAD' ,
       'TOTAL DEDUCCIONES', 'TOTAL LIQUIDACION']) #reordeno columnas, aca ya quedan en el orden que es necesario
tarjeta_nativa['DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18'] = tarjeta_nativa['Iva 10,5%']*100/10.5
tarjeta_nativa['ARANCEL'] = tarjeta_nativa['IVA 21%']*100/21            

tarjeta_nativa.insert(7, 'Iva 21 control', tarjeta_nativa['IVA 21%']/tarjeta_nativa['ARANCEL'])
                                #agrego columna de control para IVA 21%
tarjeta_nativa.insert(10, 'Iva 10,5 control', 
                                tarjeta_mastercard['Iva 10,5%']/tarjeta_mastercard['DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18'])
                                #agrego columna de control para IVA 10,5%                                
tarjeta_nativa = tarjeta_nativa.assign(Diferencia = tarjeta_nativa.iloc[:, 5:7].sum(axis=True) 
                         + tarjeta_nativa.iloc[:, 8:10].sum(axis=True)
                         + tarjeta_nativa.iloc[:, 11:18].sum(axis=True)
                         - tarjeta_nativa['TOTAL DEDUCCIONES']) #agrego columna de control final
tarjeta_nativa = tarjeta_nativa.round(2)

In [ ]:
sum_columns = [
       'Nro Liquidacion', 'VENTAS', 'ARANCEL', 'IVA 21%', 'Iva 21 control',
       'DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18', 'Iva 10,5%',
       'Iva 10,5 control', 'RETENCION IVA', 'PER B.A.I.BR.DN.01/04',
       'RETENCION IMP.GANANCIAS', 'RETENCION ING.BRUTOS',
       'RETENCION ING.BRUTOS SIRTAC', 'PERCEPCION IVA R.G. 2408   3,00 %',
       'REEMBOLSOS DE ENTIDAD', 'TOTAL DEDUCCIONES',
       'TOTAL LIQUIDACION', 'Diferencia']
max_columns = ['Nro Liquidacion']

group = {col: ('max' if col in max_columns else 'sum') for col in sum_columns}

tarjeta_nativa=tarjeta_nativa.groupby(by=['Cod Marca Producto', 'Cod Comercio']).agg(group)

In [ ]:
tarjeta_nativa = tarjeta_nativa.reset_index()

In [ ]:
tarjeta_nativa = tarjeta_nativa.drop(['Diferencia', 'Iva 21 control', 'Iva 10,5 control'], axis=1)

In [ ]:
writer = ExcelWriter('/content/drive/MyDrive/Colab Notebooks/Le Utthe/Tarjetas/10-22/Nativa1022.xlsx')
tarjeta_nativa.to_excel(writer, 'Hoja de datos', index=True)
writer.save()